In [1]:
import json
import pandas as pd
import urllib.request
import PIL
from IPython.display import Markdown
import textwrap

In [2]:
import google.generativeai as genai

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-pro-vision")

In [4]:
df = pd.read_csv('GT-Retail_personalised_product_recsys_data.csv')

In [134]:
df.iloc[11]['image_url']

'http://i.pinimg.com/400x/0d/9f/cf/0d9fcfcd20e8bd29cc2739538bfa18ea.jpg'

In [12]:
image_url = df['image_url'].iloc[0]

In [17]:
urllib.request.urlretrieve(image_url, "temp_image.jpg")  # Download the image
image = PIL.Image.open('temp_image.jpg')  # Open the image
response = model.generate_content(image)  # Generate response

In [19]:
response.text

' The model is wearing white sneakers with light blue jeans. The sneakers are low-top and have a lace-up closure. They also have a pink accent on the heel. The jeans are skinny and have a mid-rise. They are also distressed at the knees.'

In [36]:
image = image.convert('RGB')  # Example conversion to RGB format
image.save('temp_image.jpeg')

In [5]:
image_url = df.iloc[11]['image_url']
category = df.iloc[11]['category']
response = model.generate_content(
        [f"write a search description for an item from the prespective of customer with categoty:{category} for better filtering and retrieval of an item for an image", image],
        generation_config={"temperature": 0}
    )
response.text

NameError: name 'image' is not defined

In [6]:
def add_description(row):
    print(f"Image URL => {row['image_url']}")
    urllib.request.urlretrieve(row['image_url'], "temp_image.jpg")  # Download the image
    image = PIL.Image.open('temp_image.jpg')  # Open the image
    category = row['category']
    response = model.generate_content(
        [f"write a short search description for an item with categoty:{category} for an image, use as much detailing as possible for filtering and personalised recommendation", image],
        generation_config={"temperature": 0}
    )
    print(type(response))
    if response.candidates:
        # Extract text from each part and concatenate
        text_parts = [part.text for part in response.candidates[0].content.parts]
        
        # Concatenate the text from all parts
        concatenated_text = ' '.join(text_parts)
        
        # Print or use the concatenated_text as needed
        print(concatenated_text)
        print("")
        return concatenated_text
    else:
        print("No candidates in the response.")
        return ""

In [ ]:
import time
df['image_description'] = df.apply(add_description, axis=1)

Image URL => http://i.pinimg.com/400x/88/70/b0/8870b0d384138a81cfdc91f98d1dde9b.jpg
<class 'google.generativeai.types.generation_types.GenerateContentResponse'>
 These are white canvas lace-up sneakers with a pink metallic heel. They have a round toe and a flat sole. They are perfect for everyday wear and can be dressed up or down.

Image URL => http://i.pinimg.com/400x/70/d3/92/70d392390fa258ba9b5d6b83ea29857e.jpg
<class 'google.generativeai.types.generation_types.GenerateContentResponse'>
 These are women's light pink suede leather mules with a 3.5 inch heel. They have a unique criss-cross strap design over the vamp and a lightly cushioned insole. The outsole is made of rubber for traction. These mules are perfect for a casual day out or a night on the town.

Image URL => http://i.pinimg.com/400x/2c/05/db/2c05db9ff1abbcfc4bfd50dd003aac80.jpg
<class 'google.generativeai.types.generation_types.GenerateContentResponse'>
 Light blue high waisted, straight leg jeans with belt loops and fi

In [8]:
df

,user_name,age,gender,location,category,brand,price in $,click_rate,availability,ratings,image_url
0,Customer_1,63,Male,Wollongong,Shoes,Converse,66.50,164,Available,5,http://i.pinimg.com/400x/88/70/b0/8870b0d38413...
1,Customer_2,26,Male,Bunbury,Shoes,ASICS,48.00,466,Available,1,http://i.pinimg.com/400x/70/d3/92/70d392390fa2...
2,Customer_3,47,Female,Kalgoorlie-Boulder,Pants,Adidas,49.14,486,Available,1,http://i.pinimg.com/400x/2c/05/db/2c05db9ff1ab...
3,Customer_4,49,Female,Tamworth,Shirts & Tops,Adidas,41.65,392,Available,3,http://i.pinimg.com/400x/2b/2c/c3/2b2cc3a8b588...
4,Customer_5,76,Female,Devonport,Shorts,Under Armour,29.20,416,Available,2,http://i.pinimg.com/400x/a1/41/ed/a141ed588bda...
...,...,...,...,...,...,...,...,...,...,...,...
38106,Customer_7,27,Male,Canberra,Shoes,Puma,48.50,249,Available,1,http://i.pinimg.com/400x/4b/b9/1f/4bb91fc1629f...
38107,Customer_8,62,Male,Kalgoorlie-Boulder,Shorts,H&M,31.68,497,Out of Stock,4,http://i.pinimg.com/400x/29/4d/d4/294dd425f933...
38108,Customer_9,43,Female,Alice Springs,Pants,Under Armour,33.90,125,Available,5,http://i.pinimg.com/400x/33/e2/5a/33e25a1f2775...
38109,Customer_10,24,Male,Toowoomba,Shoes,Skechers,47.00,474,Available,3,http://i.pinimg.com/400x/ca/ae/2d/caae2dbcabaf...


In [11]:
df.iloc[0]['image_url']

'http://i.pinimg.com/400x/2c/05/db/2c05db9ff1abbcfc4bfd50dd003aac80.jpg'

In [119]:
df['image_description']

" I am looking for a pair of women's shoes. They should be a light pink color and have a block heel. The shoes should be made of suede and have a crisscross strap design over the toes."